In [1]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
import io
from bs4 import BeautifulSoup
import requests



import mysql.connector

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ankushpandey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ankushpandey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ankushpandey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ankushpandey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import PyPDF2

def get_page_text(url):
    return BeautifulSoup(requests.get(url).text, "html.parser").get_text()

def get_page_links(url):
    links = []
    for item in BeautifulSoup(requests.get(url).text, "html.parser").find_all('a'):
        try:
            if 'http' in item.get_attribute_list('href')[0]:
                links.append(item.get_attribute_list('href')[0])
            else:
                links.append(url + item.get_attribute_list('href')[0])
        except:
            pass
    return links

def read_pdf(loc):
    docs = os.listdir(loc)

    titles = []
    content = []
    for file in docs:
        pdfFileObj = open(loc + file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        p_cnt = 0
        try:
            text = pdfReader.getPage(p_cnt).extractText().replace('\n', ' ').replace('  ', '').lower()
            while len(text) < 1000:
                print('in loop')
                text = pdfReader.getPage(p_cnt).extractText().replace('\n', ' ').replace('  ', '').lower()
                p_cnt += 1
            else:
                titles.append(loc + file)
                content.append(text)
        except:
            pass
    return titles, content

def read_pdf_online(links):
    titles = []
    content = []
    for file in links:
        try:
            remote_file_bytes = io.BytesIO(urllib.request.urlopen(urllib.request.Request(file)).read())
            pdfReader = PyPDF2.PdfFileReader(remote_file_bytes)
            p_cnt = 0
            text = pdfReader.getPage(p_cnt).extractText().replace('\n', ' ').replace('  ', '').lower()
            while len(text) < 1000:
                print(f'in loop {len(text)}')
                text = pdfReader.getPage(p_cnt).extractText().replace('\n', ' ').replace('  ', '').lower()
                p_cnt += 1
            else:
                titles.append(file)
                content.append(text)
        except:
            text = get_page_text(file).replace('\n', ' ').replace('  ', '').lower()
            while len(text) < 1000:
                print('in loop')
                text = get_page_text(file).replace('\n', ' ').replace('  ', '').lower()
            else:
                titles.append(file)
                content.append(text)
    return titles, content




In [3]:
def preprocess(df_news):
    df_news['content']=[entry.lower() for entry in df_news['content']]
    df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True) #remove from to email 
    df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
    df_news.content =df_news.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
    df_news.content =df_news.content.replace(to_replace='-',value=' ',regex=True)
    df_news.content =df_news.content.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
    df_news.content =df_news.content.replace(to_replace='  ',value='',regex=True)                #remove double white space
    df_news.content =df_news.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace
    df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
    df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
    df_news.Subject =df_news.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
    df_news.Subject =df_news.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
    df_news.Subject =df_news.Subject.apply(lambda x:x.strip())

    for i,sen in enumerate(df_news.content):
        if len(sen.strip()) ==0:
            print(str(i))
            df_news=df_news.drop(str(i),axis=0).reset_index().drop('index',axis=1)

    return df_news


In [5]:

def tokenise(df_news):
    df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]
    return df_news



In [6]:
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
    return file_clean_k

In [8]:
def clean_more(df_clean):
    df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
    df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
    df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
    df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

    return df_clean



In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator


def create_vocabulary(df_news):
    ## Create Vocabulary
    vocabulary = set()

    for doc in df_news.Clean_Keyword:
        vocabulary.update(doc.split(','))

    vocabulary = list(vocabulary)
    return vocabulary



In [13]:
def tfidf_vector(vocabulary, df_news):
    # Intializating the tfIdf model
    tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

    # Fit the TfIdf model
    tfidf.fit(df_news.Clean_Keyword)

    # Transform the TfIdf model
    tfidf_tran=tfidf.transform(df_news.Clean_Keyword)
    ### Save model
    with open('tfid.pkl','wb') as handle:
        pickle.dump(tfidf_tran, handle)
    
    return tfidf, tfidf_tran



In [14]:
def make_model_localrepo():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ap166@jmail"
    )

    print(mydb)
    cursor = mydb.cursor()

    cursor.execute('use docsearch;')

    # cursor = mydb.cursor()

    cursor.execute('select * from datastorage;')

    extracted_items = list(cursor)

    df_news = pd.DataFrame()
    for a,b,c in extracted_items:
        if b == 3:
            loc = c
            titles, content = read_pdf(loc)

            df_temp = pd.DataFrame()
            df_temp['Subject'] = titles
            df_temp['content'] = content

            df_news = pd.concat([df_news, df_temp])

    display(df_news)

    df_news = preprocess(df_news)

    df_news = tokenise(df_news)
    
    df_clean = wordLemmatizer(df_news['Word tokenize'])#[0:10])

    df_clean = clean_more(df_clean)

    df_news.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

    df_news_save = df_news
    df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
    df_news_save.to_csv("df_news_index_local.csv", index=False, header=True)

    vocabulary = create_vocabulary(df_news)

    tfidf, tfidf_tran = tfidf_vector(vocabulary, df_news)

    with open("vocabulary_localrepo.txt", "w") as file:
        file.write(str(vocabulary))

    with open('tfid_localrepo.pkl','wb') as handle:
        pickle.dump(tfidf_tran, handle)

In [15]:
make_model_localrepo()

in loop
in loop


,Subject,content
0,Data/1814-6164-3-PB.pdf,"original article meiting chen1*, zhao wang 1*,..."
1,Data/WHO list of priority medical devices for ...,"!""#$%&'($)*$+,&),&(-$./0&12%$0/3&1/'$*),$1241/..."
2,Data/1801-6137-2-PB.pdf,original article raltitrexed as a synergistic ...
3,Data/1860-6269-2-PB.pdf,original article phase ii study of apatinib in...
4,Data/1856-6261-2-PB.pdf,reviewcurrent research on circular rnas and th...
5,Data/1837-6226-2-PB.pdf,original article vhh212 nanobody targeting the...
6,Data/1882-6312-2-PB.pdf,origingal article lncrna dpp10-as1 promotes ma...
7,Data/1799-6133-2-PB.pdf,original article nedd9 promotes cancer stemnes...
8,Data/1829-6193-2-PB.pdf,"orginal articletianqing chu1*, jun lu1*, mingh..."
9,Data/1824-6183-2-PB.pdf,original article the combination of chidamide ...


In [16]:
def make_model_remoterepo():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ap166@jmail"
    )

    print(mydb)
    cursor = mydb.cursor()

    cursor.execute('use docsearch;')

    #cursor = mydb.cursor()

    cursor.execute('select * from datastorage;')

    extracted_items = list(cursor)

    df_news = pd.DataFrame()
    for a,b,c in extracted_items:
        if b == 4:
            loc = c
            titles, content = read_pdf(loc)

            df_temp = pd.DataFrame()
            df_temp['Subject'] = titles
            df_temp['content'] = content

            df_news = pd.concat([df_news, df_temp])

    display(df_news)

    df_news = preprocess(df_news)

    df_news = tokenise(df_news)
    
    df_clean = wordLemmatizer(df_news['Word tokenize'])#[0:10])

    df_clean = clean_more(df_clean)

    df_news.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

    df_news_save = df_news
    df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
    df_news_save.to_csv("df_news_index_remote.csv", index=False, header=True)

    vocabulary = create_vocabulary(df_news)

    tfidf, tfidf_tran = tfidf_vector(vocabulary, df_news)

    with open("vocabulary_remoterepo.txt", "w") as file:
        file.write(str(vocabulary))

    with open('tfid_remoterepo.pkl','wb') as handle:
        pickle.dump(tfidf_tran, handle)

In [17]:
make_model_remoterepo()

in loop
in loop


,Subject,content
0,Data/1814-6164-3-PB.pdf,"original article meiting chen1*, zhao wang 1*,..."
1,Data/WHO list of priority medical devices for ...,"!""#$%&'($)*$+,&),&(-$./0&12%$0/3&1/'$*),$1241/..."
2,Data/1801-6137-2-PB.pdf,original article raltitrexed as a synergistic ...
3,Data/1860-6269-2-PB.pdf,original article phase ii study of apatinib in...
4,Data/1856-6261-2-PB.pdf,reviewcurrent research on circular rnas and th...
5,Data/1837-6226-2-PB.pdf,original article vhh212 nanobody targeting the...
6,Data/1882-6312-2-PB.pdf,origingal article lncrna dpp10-as1 promotes ma...
7,Data/1799-6133-2-PB.pdf,original article nedd9 promotes cancer stemnes...
8,Data/1829-6193-2-PB.pdf,"orginal articletianqing chu1*, jun lu1*, mingh..."
9,Data/1824-6183-2-PB.pdf,original article the combination of chidamide ...


In [18]:
def make_model_weblinks():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ap166@jmail"
    )

    print(mydb)
    cursor = mydb.cursor()

    cursor.execute('use docsearch;')

    #cursor = mydb.cursor()

    cursor.execute('select * from datastorage;')

    extracted_items = list(cursor)

    links = []
    for a,b,c in extracted_items:
        if b == 2:
            links.append(c)

    titles, content = read_pdf_online(links)

    df_news = pd.DataFrame()
    df_news['Subject'] = titles
    df_news['content'] = content

    display(df_news)

    df_news = preprocess(df_news)

    df_news = tokenise(df_news)
    
    df_clean = wordLemmatizer(df_news['Word tokenize'])#[0:10])

    df_clean = clean_more(df_clean)

    df_news.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

    df_news_save = df_news
    df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
    df_news_save.to_csv("df_news_index_weblinks.csv", index=False, header=True)

    vocabulary = create_vocabulary(df_news)

    tfidf, tfidf_tran = tfidf_vector(vocabulary, df_news)

    with open("vocabulary_weblinks.txt", "w") as file:
        file.write(str(vocabulary))

    with open('tfid_weblinks.pkl','wb') as handle:
        pickle.dump(tfidf_tran, handle)

In [19]:
make_model_weblinks()

,Subject,content
0,http://www.cancerbiomed.org/index.php/cocr/art...,original article heat shock protein 90 promote...
1,http://www.cancerbiomed.org/index.php/cocr/art...,breast cancer incidence and mortality in women...


In [20]:
def make_model_library():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ap166@jmail"
    )

    print(mydb)
    cursor = mydb.cursor()

    cursor.execute('use docsearch;')

    #cursor = mydb.cursor()

    cursor.execute('select * from datastorage;')

    extracted_items = list(cursor)

    df_news = pd.DataFrame()
    for a,b,c in extracted_items:
        if b == 1:
            try:
                links = get_page_links(c)[15:20]
                print(links)
                titles, content = read_pdf_online(links)
                df_temp = pd.DataFrame()
                df_temp['Subject'] = titles
                df_temp['content'] = content

                df_news = pd.concat([df_news, df_temp])
            except Exception as e:
                print(e)
                pass

    display(df_news)

    df_news = preprocess(df_news)

    df_news = tokenise(df_news)
    
    df_clean = wordLemmatizer(df_news['Word tokenize'])#[0:10])

    df_clean = clean_more(df_clean)

    df_news.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

    df_news_save = df_news
    df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
    df_news_save.to_csv("df_news_index_library.csv", index=False, header=True)

    vocabulary = create_vocabulary(df_news)

    tfidf, tfidf_tran = tfidf_vector(vocabulary, df_news)

    with open("vocabulary_library.txt", "w") as file:
        file.write(str(vocabulary))

    with open('tfid_library.pkl','wb') as handle:
        pickle.dump(tfidf_tran, handle)

In [153]:
make_model_library()

['http://www.sciepub.com/journal/ajcp', 'https://www.ncbi.nlm.nih.gov/labs/pmc/journals/1608/', 'http://www.ncbi.nlm.nih.gov/pubmed?term=%22Am%20J%20Cancer%20Res%22%5BJour%5D%20AND%20%22loattrfree%20full%20text%22%5Bsb%5D', 'http://ivyunion.org/index.php/ajcs', 'https://www.jstage.jst.go.jp/browse/acrt']


,Subject,content
0,http://www.sciepub.com/journal/ajcp,\r \tamerican journal of cancer prevention\r ...
1,https://www.ncbi.nlm.nih.gov/labs/pmc/journals...,"archive of ""american journal of cancer resear..."
2,http://www.ncbi.nlm.nih.gov/pubmed?term=%22Am%...,"""am j cancer res""[jour] and ""loattrfree full ..."
3,http://ivyunion.org/index.php/ajcs,american journal of cancer scienceuserusernam...
4,https://www.jstage.jst.go.jp/browse/acrt,annals of cancer research and therapy toggle ...
